## Distributional reinforment learning simulation for CDLab
by *Shutian @ CDLab*

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ipywidgets import interactive, fixed
plt.style.use('default')

The main content includes four parts:
- The convergence of Temporal difference(TD0)
- Why mean representation may not good
- TD Learning modification
- Dopamine Neuron firing traits

## Temporal Difference Learning

Temporal difference is the based on model-free framework to learn the state or action value. Different from Monte-carlo learning, temporal difference learning does backup immediately after arriving the next state and receiving the reward. The most famaliar formula is the Rescorla-Wagner rule which can be seen as the $TD_0$ with discount parameter set 0. For similarity, we write this formula as follows:$$ V_{t+1} = V_t + \alpha\delta_t,\  \ where\ \delta_t = r - V_t\ as\ the\ prediction\ error$$

When sample suffcient episode and set learning rate close to 0, the V will converg to the expectation of the reward distribution. Examples are as follows:

Here we use bandit task and set the reward distribution as Gaussian Distribution, we sample for 1e5 times and see the final value.

In [2]:
class bandits:
    def __init__(self, distribution, arg1,arg2):
        self.distribution = getattr(np.random, distribution)
        self.loc = arg1
        self.scale = arg2
        
    def sample(self):
        # sample one reward from the distribution
        reward = self.distribution(self.loc,self.scale)
        return reward
    
def TD_Update(V,reward,lrp,lrn):
    """ Traditional Temporal Difference Learning
    """
    V += lrp*(reward-V)
    return V 
 
globals_dict = globals()  
def PlotStateValue(distribution = 'normal',arg1=6,arg2=2,lrp = 0.01,lrn=  0.01,UpdateRule='TD_Update'):
    """Plot the estimated state value 

    Args:
        distribution (str, optional): _description_. Defaults to 'normal'.
        arg1 (int, optional): _description_. Defaults to 6.
        arg2 (int, optional): _description_. Defaults to 2.
    """
    bandit = bandits(distribution,arg1,arg2)
    trials,V = 5000, 0
    V_trajectory = [0]
    x = np.arange(0,trials+1)
    UpdateRule = globals_dict[UpdateRule]
    for _ in range(trials):
        reward = bandit.sample()
        V = UpdateRule(V,reward,lrp,lrn)
        V_trajectory.append(V)
    plt.plot(x,np.array(V_trajectory))
    plt.ylabel('Estimated Reward')
    plt.xlabel('Trials')
    
    # add horizitontal line of different distribution
    # For Gaussian, we draw the mu 
    # For Gamma, we draw mean, median and mode 
    x1 = [0,trials]
    if distribution == 'normal':
        plt.plot(x1,[arg1,arg1],color='orange',ls='--',label='Mean')
    elif distribution == 'gamma':
        # this part for TD_binary
        mean = arg1*arg2
        mode = (arg1-1)*arg2
        dis = stats.gamma(arg1,scale = arg2)
        median = dis.ppf(0.5)
        plt.plot(x1,[mean,mean],color='orange',ls='--',label='Mean')
        plt.plot(x1,[mode,mode],color='green',ls='--',label='Mode')
        plt.plot(x1,[median,median],color='red',ls='--',label='Median')
        
        # thie for quantile TD
        if lrp != lrn:
            tau = lrp/(lrp+lrn)
            quantile = dis.ppf(tau)
            plt.plot(x1,[quantile,quantile],color='purple',ls='--',label=f'{round(tau,2)} quantile')
            
    ax=plt.gca()  
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    plt.legend()
    plt.title('Estimated changes with trials')
    plt.show()

widget1=interactive(PlotStateValue, arg1=(4, 10, 1), arg2=(1, 5, 0.5),UpdateRule = fixed('TD_Update'),lrp=fixed(0.01),lrn=fixed(0.01))
widget1

interactive(children=(Text(value='normal', description='distribution'), IntSlider(value=6, description='arg1',…

## Does expectation represent everything ?

Though using traditional temporal difference learning meathod we can approximate the expectation of the reward distribution, does the expectation of the distribution mean everything? In other words, does knowing the expectation really help us make the optimal choice ?

The answer is not, please see the next example below.In this case, I will construct two distribution with the same expectation and variance. Using traditional reinforcement learning, we can converge to the same expectation. But if you think they're just as good, then you are completely wrong !

For ease of calculation, I choose distributions that have analytical solutions of mean and standard variance. For symmetrical distribution, I choose the Gaussian Distribution: $$p(x) = \frac{1}{\sqrt{2\pi}\sigma}\exp({-\frac{(x-\mu)^2}{2\sigma^2})},where\ \mu\ is\ the\ mean,\ and\ \sigma^2\ is\ the\ variance$$ 

For skewed distribution, I choose Gamma Distribution:$$p(x;\beta,\alpha) = \frac{1}{\Gamma(\alpha)\beta^{\alpha}}x^{\alpha-1}\exp(-\frac{x}{\beta}),x>0, where\ \Gamma(\alpha) = \int_{0}^{\infty}t^{\alpha-1}\exp{-t}dt$$
For Gamma Distribution, we can give the mean and variance directly: 
$$\begin{align*}
    \mu &= \alpha\beta \\
    \sigma^2 &= \alpha\beta^2 \\
    Mode &= (\alpha-1)\beta

\end{align*}
$$

Next we show the probability density distribution of Gaussian Distribution($\mu = 16,\sigma = 4\sqrt{2}$) and Gamma Distribution($\alpha = 8 ,\beta=2$), so the mean and variance of the two distribution are the same.

In [3]:
def PlotDistribution(alpha=8,beta=2):
    # set the parameter
    mu, sigma = alpha*beta,np.sqrt(alpha*beta**2)
    mode = (alpha-1)*beta

    # show the PDF
    x = np.arange(mu-20,mu+20,0.01)
    p_gaussian = stats.norm.pdf(x,mu,sigma)
    p_gamma = stats.gamma.pdf(x,alpha,scale=beta)
    PdfMode = stats.gamma.pdf(mode,alpha,scale=beta)
    PdfMean = stats.norm.pdf(mu,mu,sigma)

    # draw the PDF
    plt.plot(x,p_gaussian,label='Gaussian Distribution')
    plt.plot(x,p_gamma,label='Gamma Distribution')

    # label the mean median and mode
    plt.plot([mode,mode],[0,PdfMode],color='gray',ls='--',label='Mode')
    plt.plot([mu,mu],[0,PdfMean],color='gray',ls='-',label='Mean')
    plt.ylabel('Density')

    ax=plt.gca()  
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    plt.legend(loc = 'upper right')
    plt.title('PDF of Gaussian and Gamma Distribution')
    plt.show()
    
widget2 = interactive(PlotDistribution, alpha=(4, 10, 1), beta=(0.2, 10, 0.5))
widget2

interactive(children=(IntSlider(value=8, description='alpha', max=10, min=4), FloatSlider(value=2.0, descripti…

In [4]:
# sample enough episode and try to see which one is better
def Contrast(alpha=8,beta=2):
    mu, sigma = alpha*beta,np.sqrt(alpha*beta**2)
    iterations = 200000
    num = 0
    for i in range(int(iterations)):
        value_gaussian = np.random.normal(mu,sigma)
        value_gamma = np.random.gamma(alpha,beta)
        num += (value_gamma>=value_gaussian)
    print(num/iterations)
widget3 = interactive(Contrast, alpha=(4, 10, 1), beta=(0.2, 10, 0.5))
widget3

interactive(children=(IntSlider(value=8, description='alpha', max=10, min=4), FloatSlider(value=2.0, descripti…

Simulations are as above and we can see that the probability of gamma reward better than gaussian reward is lower than 0.5, which means that given the same expectation and variance we can't make the best choice and we need to know more information about the distribution. And the full mathmetical proof of why gaussian is better than gamma is as follow and I try to give the closed form solution.

## TD-Learning modification

In this part, I will give some formats of new TD learning and the modifications are mainly focused on learning rate and prediction error. I refer to the [Lowet et al., 2020] and give 3 types of updated rule. Details and the effect of updated rule are combined with example.

(1) Binarize rule
This update method will converge to the median of the distribution. The full mathematical proof is not given here but we can imagine when this equation gets to the equilibrium. If this equation gets the balance which means that the estimated V jitters around a certain value and the prediction error sometimes positive and sometimes negative with the equal probability. So the certain value must be the median of the distribution.

$$V = 
V + \alpha
\begin{cases}
-1 \ \text{  if } \delta \leq 0 \\
1 \ \text{  if } \delta \gt 0
\end{cases}
$$

Considering the distinction between the expectation and the median of one distribution，I still take the skewed distribution-gamma distribution for example. Though no clear closed form solution of median, python scipy moudle provides the `stats.gamma.ppf` function to do value approximation.

In [5]:
# Binarize rule simulation

# update rule
def TD_UpdateBinary(value,reward,lrp,lrn):
    RPE = reward - value
    error = -1 if RPE <= 0 else 1
    value += lrp*error
    return value

widget4 = interactive(PlotStateValue,distribution='gamma',arg1=(4, 10, 1), arg2=(1, 5, 0.5),UpdateRule = 'TD_UpdateBinary',lrp=fixed(0.01),lrn=fixed(0.01))
widget4

interactive(children=(Text(value='gamma', description='distribution'), IntSlider(value=6, description='arg1', …

(2) Seperate learning rate but error binary(quantitle regression): In this update rule, we add variability in the learning rate and set two separate learning rate, in which $\alpha^+$ for positive prediction error and $\alpha^-$ for negative prediction error. Its formula is bolow:
$$V = 
V + 
\begin{cases}
\alpha^-*(-1) \ \text{  if } \delta \leq 0 \\
\alpha^+*(+1) \ \text{  if } \delta \gt 0
\end{cases}
$$

For any combination of $\alpha^+$ and $\alpha^-$, a value predictor that learns according to the above rule will converge to the $\frac{\alpha^+}{\alpha^++\alpha^-}=\tau$=$\tau$-th quantile. For example, when $\alpha^+$ = $\alpha^-$, it converges to the median of the distribution. If you want to see the mathematic details, please look forward to the [quantile regression](https://en.wikipedia.org/wiki/Quantile_regression)

In [6]:
def TD_UpdateQuantile(value,reward,lrp,lrn):
    RPE = reward - value
    error = -1 if RPE <= 0 else 1
    if error == 1:
        value += lrp*error
    else:
        value += lrn*error
    return value

widget5 = interactive(PlotStateValue,distribution='gamma',arg1=(4, 10, 1), arg2=(1, 5, 0.5),lrp=(0.01,0.1,0.01),lrn=(0.01,0.1,0.01),UpdateRule = 'TD_UpdateQuantile')
widget5

interactive(children=(Text(value='gamma', description='distribution'), IntSlider(value=6, description='arg1', …

(3) Seperate learning rate and prediction error(Expectile regression):One can also consider a family of value predictors that retains the original form of the update rule and with different learning rate. Actually there exists this kind of update rule as follows:
$$V = 
V + 
\begin{cases}
\alpha^-*\delta \ \text{  if } \delta \leq 0 \\
\alpha^+*\delta \ \text{  if } \delta \gt 0
\end{cases}
$$

This update rule gives rise to a range of value estimates called expectiles, which generalize the mean just as quantiles generalize the median. However, unlike quantiles, expectiles do not bear a straightforward relationship to the CDF. To understand them, it is necessary to adopt a more general perspective on learning. For math details, please refer to [expectile regression](https://epub.ub.uni-muenchen.de/31542/1/1471082x14561155.pdf)

In [7]:
def TD_UpdateExpectile(value,reward,lrp,lrn):
    RPE = reward - value
    if RPE >0:
        value += lrp*RPE
    else:
        value += lrn*RPE
    return value

widget5 = interactive(PlotStateValue,distribution='gamma',arg1=(4, 10, 1), arg2=(1, 5, 0.5),lrp=(0.01,0.1,0.01),lrn=(0.01,0.1,0.01),UpdateRule = 'TD_UpdateExpectile')
widget5

interactive(children=(Text(value='gamma', description='distribution'), IntSlider(value=6, description='arg1', …

## Dopamine Neuron Simulation

In this part I will simulate the dopamine neuron spking neural dynamics when encourting the reward for the reason to illustrate why they will response to the cue aftering learning the correlation between cue and reward. Most of us know that the dopamine may reflect the reward prediction error just like the schulz et al., 1997 shows. However we may ignore the intense spiking after the cue which predicting the reward in the near future.

<p align="center">
  <img src="pictures/dopamine1997.png" alt="dopamine1997">
</p>

Most of us just focus on the Temporal Difference rule but we miss the math detail of the TD learning in dopamine coding. Next I will reveal the math details and show how the TD error does backpropagation.

$$\delta(t) = r(t) + \gamma\hat{V}(t+1)-\hat{V}(t)$$
$$\hat{V}(t) = \sum_iw_ix_i(t)$$
$$\Delta w_i = \alpha \sum_t x_i(t)\delta(t)$$

In [9]:
def PlotTDError(trials=1,lrp=0.01,lrn=0.01,gamma=1,magnitudes=[10],dis = False):
    """Plot the TD error backpropagation

    Args:
        trials (int, optional): _how many trials the neuron does_. Defaults to 1000.
        lrp (float, optional): _positive learning rate_. Defaults to 0.01.
        lrn (float, optional): _negative learning rate_. Defaults to 0.01.
        gamma (int, optional): _discount parameter_. Defaults to 0.
        magnitude (int, optional): _the reward magnitude_. Defaults to 2.
    """
    # Here we set time length for 30, what happens in the 30 seconds are as follows
    # 0-4 nothing; 5:14 cue; 15-24 delay; 25 reward; 26-34 nothing
    stimulus,R = np.zeros(35),np.zeros(35)
    is_delay = np.zeros(35)
    is_delay[15:26] = 1 
    stimulus[5:14] = 1

    # initialize the state value and weight
    for magnitude in magnitudes:
        V = np.zeros(36)
        E = np.zeros(35)
        for i in range(trials):
            if dis == False:
                R[25] = magnitude
            else:
                R[25] = np.random.normal(magnitude,2)
            if lrp == lrn:
                for t in range(35):
                    if is_delay[t] == 1:
                        E[t] = R[t]+gamma*V[t+1]-V[t]
                        V[t] += lrp*E[t] 
            else:
                for t in range(35):
                    if is_delay[t] == 1:
                        E[t] = R[t]+gamma*V[t+1]-V[t]
                        if E[t] <= 0:
                            V[t] += lrn*E[t]
                        else:
                            V[t] += lrp*E[t]     
        x = np.arange(35)
        plt.plot(x,E,label=f'magnitude = {magnitude}')
    
    #draw pictures
    ax=plt.gca()  
    plt.title('TD error changes with time')
    plt.xlabel('Time')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    plt.legend()
    plt.show()
widget7 = interactive(PlotTDError,lrp=(0.01,0.1,0.01),lrn=(0.01,0.1,0.01),trials=(1,800,10),gamma=fixed(0.99),magnitudes=fixed([10,20]))
widget7   
    

interactive(children=(IntSlider(value=1, description='trials', max=800, min=1, step=10), FloatSlider(value=0.0…

Question: How to combine dopamine neuron firing traits with the distributional reinforment learning algorithm? In other words, how can we judge that the dopamine neuron do the distributional coding ?